# Data structure interface and matrix algebra examples
Felix Zaussinger | 13.11.2020

## Core Analysis Goal(s)
1. Settle for a data structure that can well represent the playing field at all times of the game.
2. Find a suitable data structure for defining the "plots" or "blocks" on which
players can manage their land
3. Define the interface: what are the dimensions (x, y, z)

## Key Insight(s)
1. Numpy masked arrays (np.ma) are perfectly fine. We should use those.
2. A simple numpy function (np.block) may be enough to work with these blocks

## Explanations
1. in python, numpy matrix algebra should always be preferred to hard-coded for-loops
2. Python itself can be quite slow, which is why numpy is written in a lower-level
languages (i think the most prominent one is written in C). If you use the numpy matrix computations, for-loops still need to be
used to calculate things, but they will be executed in C, which is MUCH faster.
This is the crucial difference. Does not matter much for 80 x 80 matrix,
but I would wish to not close the door to being able to the game on a 800 x 800
matrix as well! (:

In [46]:
import os
import numpy as np
import numpy.ma as ma

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("ticks")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define size of playing field as a function of the number of plots

In [47]:
n_plots = 4
rows = cols = 80

# check if playing field size and plot number works together
try:
    # modulo must be zero
    assert rows % n_plots == 0
except AssertionError as e:
    print("playing field size and plot number dont work together. field size must be divideable by plot number without rest.")

Create a dummy "plot" matrix

In [48]:
plot_length = int(rows/n_plots)
dummy_plot = np.ones(plot_length**2).reshape((plot_length, plot_length))
dummy_plot.shape

(20, 20)

Create a few meaningful block matrixes

Example matrix:

  1 2

A x y

B z x

In [49]:
A1 = dummy_plot
A2 = dummy_plot * 2
B1 = dummy_plot * 3
B2 = dummy_plot * 4

Create a bigger (40 x 40) matrix from the 4 smaller block matrices

In [50]:
large_matrix = np.block([[A1, A2], [B1, B2]])
large_matrix

array([[1., 1., 1., ..., 2., 2., 2.],
       [1., 1., 1., ..., 2., 2., 2.],
       [1., 1., 1., ..., 2., 2., 2.],
       ...,
       [3., 3., 3., ..., 4., 4., 4.],
       [3., 3., 3., ..., 4., 4., 4.],
       [3., 3., 3., ..., 4., 4., 4.]])

In [51]:
large_matrix.shape

(40, 40)

Works like a charm! We can now distinguish the various plots on our playing field
by using np.block.

Now, let's figure out how to manipulate the blocks separately. For this, I found
some suggestions here: https://stackoverflow.com/questions/39104999/multiply-each-block-of-a-block-matrix-with-different-coefficient-in-numpy.
I will follow those and will try to use numpy's Kronecker Delta function to solve our problem.

1. The number of plots is determined by $n_{plots}$, in our case, $n_{plots} := 4$.

2. Assuming our playing field is a $(n * n)$ quadratic matrix, we also need to define $n$. For now, let $n := 80$.

3. We can define $length_{plot}$ as $\frac{n_{plots}}{n}$, in our case, $length_{plot} = 20$.

4. For updating the blocks/plots separately, we need to define a $n_{plots} * n_{plots}$ coefficient matrix
and from thereon construct a $(length_{plot} * n) * (length_{plot} * n)$ block coefficient matrix.

5. We can then exploit the Kronecker Delta function, and multiply the small, $n_{plots} * n_{plots}$ coefficient matrix
with a $length_{plot} * length_{plot}$ unit matrix (create with np.ones).

6. In the end, we use this $(length_{plot} * n) * (length_{plot} * n)$ block coefficient matrix
to update our $(n * n)$ playing field matrix.

Et voila! Is this understandable?

In [52]:
large_dummy_matrix = np.ones_like(large_matrix)
large_dummy_matrix.shape

(40, 40)

In [53]:
n = plot_length
coef_matrix = np.array([[1, 2], [3, 4]])

result = np.multiply(large_dummy_matrix, np.kron(coef_matrix, np.ones((n,n))))
result


array([[1., 1., 1., ..., 2., 2., 2.],
       [1., 1., 1., ..., 2., 2., 2.],
       [1., 1., 1., ..., 2., 2., 2.],
       ...,
       [3., 3., 3., ..., 4., 4., 4.],
       [3., 3., 3., ..., 4., 4., 4.],
       [3., 3., 3., ..., 4., 4., 4.]])

Again, works like a charm! Let's now put everything together in a neat data
structure tailored to our playing field problem.

Unfortunately, mixing characters (A-D) and numbers (1-4) is pretty bad
for the coding team for many reasons, so let's please just use numbers
(just like in linalg courses) to specify plots/blocks on our map.

Both Rows and Columns are characterised by numbers from 1-4 which when combined
uniquely identify each block/plot as follows:


| - | 1   | 2   | 3   | 4  |
--- | --- | --- | --- | ---
| 1 | 11  | 12  | 13  | 14 |
| 2 | 21  | 22  | 23  | 24 |
| 3 | 31  | 32  | 33  | 34 |
| 4 | 41  | 42  | 43  | 44 |

Hope that is fine - thanks!

Use index matrix and value concatenation to dynamically create plot/block
definitions based on playing field size and plot number (ideally, we want to
stay flexible here).

https://stackoverflow.com/questions/36700914/how-to-merge-2-numpy-arrays-and-concatenate-their-values

In [54]:
n_plots
plot_length
n = rows

In [55]:
# define blocks/plots via unique integers
matrix_indizes = np.indices((n_plots, n_plots), dtype="uint8") + 1
row_indizes, column_indizes = matrix_indizes[0], matrix_indizes[1]
plot_definition_matrix = np.char.add(row_indizes.astype(np.str), column_indizes.astype(np.str)).astype(np.uint8)
plot_definition_matrix

array([[11, 12, 13, 14],
       [21, 22, 23, 24],
       [31, 32, 33, 34],
       [41, 42, 43, 44]], dtype=uint8)

Now, let's use the small "plot_definition_matrix" to create the definitions
for the real world, large playing field matrix consisting of many small block
matrices.

In [56]:
dummy_playing_field_matrix = np.ones(shape=(rows, cols), dtype=np.uint8)
dummy_playing_field_matrix.shape

(80, 80)

In [57]:
large_plot_definition_matrix = np.multiply(
    dummy_playing_field_matrix,
    np.kron(plot_definition_matrix, np.ones(shape=(plot_length, plot_length)))
)

large_plot_definition_matrix

array([[11., 11., 11., ..., 14., 14., 14.],
       [11., 11., 11., ..., 14., 14., 14.],
       [11., 11., 11., ..., 14., 14., 14.],
       ...,
       [41., 41., 41., ..., 44., 44., 44.],
       [41., 41., 41., ..., 44., 44., 44.],
       [41., 41., 41., ..., 44., 44., 44.]])

In [58]:
large_plot_definition_matrix.shape

(80, 80)

## Bringing it all together

Let's now define the common, underlying data structure/interface needed to
play our game. It consists of a set of matrices, which in turn can (but don't
necessarily need) be combined ("stacked") into a 3D array representation ("data cube").

To be flexible, I would suggest we stick with 2D arrays for now as they are
easier to change/adjust/adapt and less prone to errors. I would only transition
towards a 3D representation later on, given there is time and motivation to do so.

We probably need the following 2D matrices/arrays of equal shape (same number of
rows and columns, always!!) to be able to play our game. Note that all of them
should be numpy masked arrays (np.ma), because we crop out all LULC data
outside of our area of interest (the biosphere park boundary).

This table summarizes all parameters needed to describe the game's playing field,
(given that a raster map describing the LULC already exists).

Parameter | Data structure | Data type | Shape | Description
--- | --- | --- | --- | ---
n_pixels | int | np.uint16 | (1,) | The size of the quadratic playing field along one dimension, where n_pixels = rows = cols.
n_blocks | int | np.uint8 | (1,) | The number of plots/blocks to be created.

From these initial parameters, we create a few more "derived" parameters:

Parameter | Data structure | Data type | Shape | Description
--- | --- | --- | --- | ---
n_pixels_block | int | np.uint8 | (1,) | n_blocks / n_pixels


This table summarizes all matrices necessary for creating the game's back-end:

Matrix | Variable name | Data structure | Data type | Shape | General description | Mapping description
--- | --- | --- | --- | --- | --- | ---
**Land-use and land-cover (LULC) types** | lulc_matrix | np.ma | np.unit8 | (n_pixels, n_pixels) | defines the land-cover and land-use types of the playing field | each integer maps to a unique LULC class.
**Plot/Block Definitions** | plot_definition_matrix | np.ma | np.unit8 | (n_blocks, n_blocks) or (n_pixels, n_pixels) |  defines the plots/blocks the players can manipulate | each integer maps to a unique plot/block identifier. this "small" matrix is brought into (n_pixels, n_pixels) shape via the kronecker delta function.
**Cooperation/Teamwork** | cooperation_matrix | np.ma | np.bool | (n_pixels, n_pixels) |  defines in which block players from a certain stakeholder group are open for cooperation with players from other stakeholder groups.
**Tourism/SSDA** | tourism_matrix | np.ma | np.bool | (n_pixels, n_pixels) |  defines the plot which SSDA designates as being particularly valuable for tourism based on biodiversity, etc.
